# Demo

In [1]:
from SmartContractAuthority.sca import SmartContractAuthority
from University.university import University
from Credential.credential import Credential
from utils.identifiers_utils import *
from utils.file_utils import *
from Student.student import Student
import json
from SimulationUtils.simulationUtils import *

SCA_PATH = './smart_contract_authority/'
UNIVERSITIES_PATH = './universities/'
STUDENTS_PATH = './students/'
CONTRACTS_BUILD_PATH = './SmartContracts/build'

## Creazione Smart Contract Authority
Viene associato un account eth alla Smart Contract Authority che fa il deploy dei propri contratti

In [2]:
SmartContractAuthority.create_sca(SCA_PATH, CONTRACTS_BUILD_PATH)

SmartContractAuthority deployed at address: 0x43902a9a9FFfd5f57A87690E44873b140Bb463bF
SCA creata con successo e configurazione salvata.


### Creazione dell'istanza
Viene effettivamente caricata l'istanza della SCA a partire dal file JSON di persistenza generato nel blocco precedente

In [3]:
sca = SmartContractAuthority(SCA_PATH+"SCA", CONTRACTS_BUILD_PATH)

## Creazione Università
In questo blocco vengono create le università della rete assegnando loro un nome, che poi risulterà nelle directory dedicate, ed un account eth tramite il quale faranno il deploy dei contratti per la gestione di credenziali e studenti.

In [4]:
University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

SIDSmartContract deployed at: 0x2DE71C39CB20C6D4cc648442656c21A59d5713f4
CIDSmartContract deployed at: 0x5BdBB86D3D7A4370D8dD6fcB4E4bdBcF979ea8B0
Università creata con successo.
SIDSmartContract deployed at: 0x3989999C00976266C2Ec3a8c5152CfBd04F27686
CIDSmartContract deployed at: 0x8fFceCeB74a34aa34F5312C8C4a0FB3Bc366C1d6
Università creata con successo.


### Creazione dell'istanza
Viene effettivamente caricata l'istanza delle università a partire dal file JSON di persistenza generato nel blocco precedente

In [5]:
uni_origin = University(UNIVERSITIES_PATH+'University_UniSalerno', CONTRACTS_BUILD_PATH)

uni_erasmus = University(UNIVERSITIES_PATH+'University_UniCaen', CONTRACTS_BUILD_PATH)

## Creazione studente
Infine, viene creato uno studente che, a scopo dimostrativo, verrà immatricolato presso un'università che poi verrà considerata di origine per poi iniziare un'esperienza erasmus presso l'altra università creata nella fase precedente.

In [6]:
Student.create_student(STUDENTS_PATH)


Studente creato e salvato in: ./students/Student_AntonioCarbone/persistency/student_data.json


### Creazione dell'istanza
Viene effettivamente caricata l'istanza dello studente a partire dal file JSON di persistenza generato nel blocco precedente

In [7]:
studente = Student(STUDENTS_PATH+'Student_AntonioCarbone')

Studente 'Antonio Carbone' caricato da ./students/Student_AntonioCarbone


## Accreditamento università create
<p>Affiché possano operare all'interno del sistema come fornitori di credenziali, le università, devono essere accreditate da parte della SCA (paragrafo 2.2.1).<p>
<p>In questo modo la chiave pubblica e gli indirizzi degli smart contract delle università sono registrati sulla blockchain ed abbinati ad un UID che viene deciso al momento della registrazione dalla SCA ed assegnato alle università.<p>

In [8]:
#1. registrazione dell'università sulla blockchain.
UID, sca_contract_address = sca.register_university(uni_origin) 

#2. assegnazione dell'UID.
uni_origin.update_uid(UID)

#3. rilascio dell'indirizzo del registro delle università (smart contract),
#posseduto dalla SCA, per future rischieste di verifica.
uni_origin.update_sca_contract_address(sca_contract_address)     
                                                               
UID, sca_contract_address = sca.register_university(uni_erasmus)
uni_erasmus.update_uid(UID)
uni_erasmus.update_sca_contract_address(sca_contract_address)

Registrazione Università (UID: UNISA) completata con successo. Transaction Hash: 212f45b4d7bd5f8f0b456cf2435913763f9823952c31d1ac86328450304cb165
Registrazione Università (UID: ENSICAEN) completata con successo. Transaction Hash: a2d7a1e314ef68f3a49d83c79362b3ac763cba3983a73e912cd25296a832c829


## Immatricolazione presso università di origine
Implementazione dello scenario descritto nel paragrafo 2.3.1 della documentazione, lo studente si immatricola presso l'università di origine ed ottiene una credenziale studente valida all'interno della rete gestita dalla SCA.

In [9]:
#1. Registrazione della chiave pubblica dello studente e rilascio del SID.
credenziale_studente, sid = uni_origin.register_student(studente)

#2. assegnazione del SID allo studente.
studente.set_sid(sid) 

#3. rilascio della credenziale di immatricolaizone allo studente, che la aggiunge alle proprie.
studente.save_credential(credenziale_studente)

Registrazione SID completata con successo. Transaction Hash: 9dff30c5857b1114cf8c26750273bca4e72c29348f8970aa164e1fea39bf99af
Registrazione CID completata con successo. Transaction Hash: 04c3684cf0d767a004cfb027aa1b47f3aaaef4acf42ffe2f1fc31aedf07ea319
Credenziale salvata in: ./students/Student_AntonioCarbone/persistency/credentials/credential_0.json


## Immatricolazione presso università estera
Implementazione dello scenario descritto nel paragrafo 2.3.2 della documentazione, lo studente presenta la propria credenziale rilasciata dall'università d'origine presso l'università in cui desidera effettuare l'erasmus la quale salva il SID ed il CID dello studente dopo aver controllato che tutte le informazioni ricevute siano in regola.

In [10]:
#1. immatricolazione presso l'università estera che, verifica la validità della credenziale fornita dallo studente,
#verifica la firma inserita dall'università che ha rilasciato la credenziale ed infine propone una challenge allo studente.
uni_erasmus.register_erasmus_student(studente, studente.credentials[0].toJSON())

#2. L'università estera aggiorna la lista di studenti in erasmus presso la propria struttura.
uni_erasmus.update_erasmus_students(studente, credenziale_studente.CID)

Il CID associato alla credenziale fornita risulta valido.
Firma dell'issuer verificata correttamente.
Il SID contenuto nella credenziale è valido.
Firma dello studente verificata correttamente.
Complimenti Antonio Carbone 
Immatricolazione Erasmus completata!
UniCaen ti dà il benvenuto!


## Richiesta e rilascio credenziale carriera
<p>Al termine dell'esperienza all'estero dello studente, esso può richiedere il rilascio della propria credenziale carriera attestante tutto ciò che ha fatto in erasmus.<p>
<p>Per fare ciò lo studente deve presentare il proprio SID attraverso una richiesta specifica per il rilascio (da noi simulata attraverso request_career_credential()). L'università estera, invece, dovrà verificare validità della credenziale in suo possesso, firma dell'università d'origine e challenge per poi rilasciare la credenziale firmata da lei.<p>

In [11]:
#1. richiesta della credenziale inviando il proprio SID, 
# l'università verifica tutto ciò che ha ricevuto e rilascia la career_credential
career_credential = uni_erasmus.request_career_credential(studente)

#2. rilascio della credenziale carriera allo studente, che la aggiunge alle proprie.
studente.save_credential(career_credential)

Credenziale di immatricolazione dello studente valida.
Il SID fornito è valido.
Challenge di autenticazione superata.
Registrazione CID completata con successo. Transaction Hash: e4076f2cf60905e5e968c1c2487477187333fcd154dc245db590ac1cb24895cd
Complimenti Antonio Carbone 
Richiesta della tua credenziale carriera completata!
Arrivederci da UniCaen!
Credenziale salvata in: ./students/Student_AntonioCarbone/persistency/credentials/credential_1.json


## Costruzione della credenziale parziale
<p>Questo meccanismo è stato realizzato puramente a scopo dimostrativo, dal momento che sorge la necessità di creare una credenziale da condividere.<p>
<p>Perciò è stato creato un wizard messo a disposizione dello studente tramite il quale può scegliere quale delle credenziali in suo possesso condividere e quali properties includere nella condivisione.<p>

In [12]:
#1. selezione della credenziale da condividere e selezione delle properties da divulgare,
#preoccupandosi chiaramente di gestire il meccanismo delle proofs.
partial_credential = studente.build_shared_credential()

#2. rilascio della credenziale parziale allo studente, che la aggiunge alle proprie.
studente.save_credential(partial_credential)


Seleziona una credenziale da condividere:
0: CID = CID:UNISA:1
1: CID = CID:ENSICAEN:1

Seleziona le proprietà da condividere (y/n):

Credenziale parziale costruita con successo:
{
    "certificateId": "CID:ENSICAEN:1",
    "studentId": "SID:UNISA:1",
    "universityId": "UID:ENSICAEN",
    "issuanceDate": "2025-07-20",
    "properties": [
        {
            "typology": "ErasmusInfo",
            "data": {
                "programName": "Erasmus+ Europe",
                "startActivity": "2024-09-01",
                "endActivity": "2025-01-31"
            },
            "nonce": "17530057935618168",
            "merkle_proof": [
                [
                    "fadff90d8c35cd331e7438acc8ba5246c5349ffd62eb14a6a205c0ea746d6fce",
                    "R"
                ],
                [
                    "b89ae0d2f48110e6e68b096ca23b0a01e559ff3d092f6f85837a608e7c467bac",
                    "R"
                ],
                [
                    "214a4475b76f9e329caa6

## Condivisione credenziale
Infine, la credenziale appena creata viene condivisa (paragrafo 2.3.4) all'università d'origine, che ne può verificare la validità attraverso il meccanismo delle merkle proofs allegate ad ogni property.

In [13]:
uni_origin.validate_shared_credential(studente,partial_credential.toJSON())

L'università che ha rilasciato la credenziale condivisa è fidata.
Il SID presente nella credenziale è valido.
Challenge di autenticazione superata.
Le property sono state verificate correttamente!
La credenziale condivisa è valida


## Revoca
<p>Tutto ciò che nelle fasi precedenti è stato registrato e rilasciato può, come richiesto da specifica, essere revocato.<p>
<p>Questo viene fatto sempre tramite blockchain, in particolare tramite richieste effettuate da parte dell'università che ha rilasciato la credenziale verso il proprio smart contract dedicato.<p>
<p>A scopo illustrativo questo procedimento viene effettuato solo per le credenziali, ma sono predisposti tutti i metodi di modifica di SID ed UID<p>

### Revoca credenziale studente

In [ ]:
uni_origin.revoke_cid("CID:UNISA:1")

### Revoca credenziale carriera

In [ ]:
uni_erasmus.revoke_cid("CID:ENSICAEN:0")

# Threat models

## Manomissione di una credenziale
<p>In questo blocco, tramite l'ausilio di una funzione che manomette il JSON di una credenziale, viene testata la resistenza della soluzione al threat model 1.3.2 e, implicitamente, anche al threat 1.3.3.<p>
<p>Difatti, la credenziale modificata, genera una condizione di eccezione poichè viene verificata una incongruenza nelle merkle proof (si tratta di una credenziale parziale)<p>
<p>Nel caso di una credenziale totale, invece, la situazione è del tutto simile, come possiamo osservare nel secondo snippet di codice. Viene infatti rilevata l'invalidità della firma.<p>

In [ ]:
parsed_partial_credential = json.loads(partial_credential.toJSON())
tampered_credential = tamper_all_credential_properties(parsed_partial_credential)
uni_origin.validate_shared_credential(studente, json.dumps(tampered_credential, ensure_ascii=False, indent=4))

In [ ]:
parsed_partial_credential = json.loads(studente.credentials[1].toJSON()) #la credenziale 1 nella demo è quella che lo studente ha ottenuto al termine della carriera ERASMUS
tampered_credential = tamper_all_credential_properties(parsed_partial_credential)
uni_origin.validate_shared_credential(studente, json.dumps(tampered_credential, ensure_ascii=False, indent=4))

## Ente certificatore fasullo
<p>In questo blocco viene testata la resistenza del sisitema al threat model 1.3.4, in particolare un ente certificatore che è accreditato presso la SCA ma fornisce sotto richiesta di uno studente credenziali contenenti informazioni fasulle.<p>
<p>Chiaramente se la credenziale fasulla viene registrata sulla blockchain e le informazioni contenute in essa sono in regola non è possibile evitare questa situazione tramite l'implementazione della soluzione. Bensì dovrebbe essere un terzo che si rende conto della situazione e segnala l'ente e gli studenti complici alla SCA che provvederà a revocare l'UID in questione e comunicherà alle università di origine degli studenti coinvolti di revocarne i sid.<p>
<p>Allo stesso modo anche il threat model dell'ente certificatore malintenzionato (1.3.5) che modifica le credenziali rilasciate

In [14]:
University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)
uni_fake = University(UNIVERSITIES_PATH+'University_UniFake', CONTRACTS_BUILD_PATH)

#Accreditamemnto dell'università fasulla
UID, sca_contract_address = sca.register_university(uni_fake) 
uni_fake.update_uid(UID)
uni_fake.update_sca_contract_address(sca_contract_address) 

Student.create_student(STUDENTS_PATH)
studente_complice = Student(STUDENTS_PATH+'Student_CompliceComplice')

#Immatricolazione dello studente complice presso un'università accreditata
credenziale_studente_complice, sid = uni_origin.register_student(studente_complice)
studente_complice.set_sid(sid)
studente_complice.save_credential(credenziale_studente_complice)

#Immatricolazione presso l'università fasulla per il rilascio della credenziale fake
uni_fake.register_erasmus_student(studente_complice, studente_complice.credentials[0].toJSON())
uni_fake.update_erasmus_students(studente_complice, credenziale_studente_complice.CID)

#Richiesta della credenziale carriera fasulla
fake_career_credential = uni_fake.request_career_credential(studente_complice)
studente_complice.save_credential(fake_career_credential)

#Revoca di UID e del SID successivamente alla segnalazione
sca.revoke_uid(uni_fake.UID)
uni_origin.revoke_sid(studente_complice)

SIDSmartContract deployed at: 0xeC27Bbf781Ef45277a8E67d3e44e4df0f9256a8b
CIDSmartContract deployed at: 0x97C62e5E2C4c324f1F8B920926A38939Cc1C29ff
Università creata con successo.
Registrazione Università (UID: UNIFAKE) completata con successo. Transaction Hash: 6dded99076f52c7f1da4f2711ef469053f7516fe755e31b4c3ef0ce6b2cd2c03

Studente creato e salvato in: ./students/Student_CompliceComplice/persistency/student_data.json
Studente 'Complice Complice' caricato da ./students/Student_CompliceComplice
Registrazione SID completata con successo. Transaction Hash: 95a54e6b6c7ea51f0ba417f060a3e2608f0162da4e32645824525e2e8a443b66
Registrazione CID completata con successo. Transaction Hash: e58aa83c19152e3f148d3ff49cf77f554c0ea707c7e922fdf6f2eed59c22add3
Credenziale salvata in: ./students/Student_CompliceComplice/persistency/credentials/credential_0.json
Il CID associato alla credenziale fornita risulta valido.
Firma dell'issuer verificata correttamente.
Il SID contenuto nella credenziale è valido.

In [20]:
from University.services.university_blockchain_manager import UniversityBlockchainManager
manager = UniversityBlockchainManager('http://127.0.0.1:7545')

origin_uni_uid = split_UID(uni_origin.UID)
_, complice_sid = split_SID(studente_complice.SID)

_, _, isValid = manager.verify_sid_on_chain(uni_origin.sca_contract_instance, origin_uni_uid, complice_sid)
if not isValid:
    print('SID non valido')


SID non valido


In [21]:
#Prova di immatricolazione di uno studente affidabile presso l'università fasulla per verifica revoca dell'UID.
new_fake_credential,_ = uni_fake.register_student(studente)
uni_origin.validate_shared_credential(studente, new_fake_credential.toJSON())

Registrazione SID completata con successo. Transaction Hash: a71e4aacec8801b622d0aa526056b0be6af28a5a6b71a0f891334bf505ddf970
Registrazione CID completata con successo. Transaction Hash: e4c2ebf41d7446e02aff2539bdaf7c19a1591a8d984742e01614af4420767876


Exception: Errore nella verifica del'UID dell'università issuer: L'università che ha rilasciato la credenziale condivisa non è più fidata.

## Sosia
<p>Per simulare lo scenario di un sosia (Threat 1.3.6), proviamo a far condividere la credenziale dello studente dei precedenti esperimenti ad un sosia, immatricolando quest'ultimo alla stessa università, anche se questo non influenza l'esito dell'esperimento.<p>
<p>Lo studente difatti, non riesce a superare la challenge di autenticazione, e viene pertanto generata una eccezione sul controllo della firma del nonce.<p>

In [ ]:
Student.create_student(STUDENTS_PATH)
studente_sosia = Student(STUDENTS_PATH+'Student_SosiaSosia')
credenziale_studente_sosia,sid_sosia = uni_origin.register_student(studente)
studente_sosia.set_sid(sid_sosia)
studente_sosia.save_credential(credenziale_studente_sosia)

In [ ]:
uni_origin.validate_shared_credential(studente_sosia,studente.credentials[1].toJSON())

## Forgiatore di credenziali
<p>In questo blocco, tramite l'ausilio di un metodo che crea una nuova credenziale carriera, viene testata la resistenza della soluzione al threat model 1.3.7.<p>
<p>Lo stesso metodo viene utilizzato dall'università estera quando le viene chiesto il rilascio di una nuova credenziale carriera, ma in questo caso la registrazione non viene effettuata dato che l'ente certificatore non è stato accreditato presso la SCA.<p>

In [ ]:
forged_properties = generate_random_properties(5)
forged_credential = Credential(
    certificateId = assemble_CID('UNIFORGE', 7),
    studentId="SID:UNISA:1",
    universityId="UID:UNIFORGE",
    issuanceDate= data_odierna(),
    properties=forged_properties
)
uni_origin.validate_shared_credential(studente, forged_credential.toJSON())